### Machine translation Korean to English
##### 데이터출처: AiHub 한-영 말뭉치 데이터

In [4]:
import collections
from jupyter_helpers import *
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [5]:
# GPU check
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18378265822269684514
]


#### 1. 데이터 불러오기
#### 구어체의 한-영 말뭉치 데이터를 불러왔다.

In [6]:
import os, sys, time
import pandas as pd
import numpy as np

df1 = pd.read_excel('C:/Users/mark/Desktop/machine_translator_project/data/1_구어체(1)_200226 (1).xlsx')
df2 = pd.read_excel('C:/Users/mark/Desktop/machine_translator_project/data/1_구어체(1)_200226.xlsx')

df = pd.concat([df1,df2])
print(df.info()) # 400,000 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   SID     400000 non-null  int64 
 1   원문      400000 non-null  object
 2   번역문     400000 non-null  object
dtypes: int64(1), object(2)
memory usage: 12.2+ MB
None


In [7]:
kor = df['원문'].tolist()
eng = df['번역문'].tolist()
#print(len(kor))
#print(len(eng))

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re

#### 데이터 확인

In [9]:
for sample_i in range(2):
    print('small_vocab_en {}번째 줄:  {}'.format(sample_i + 1, kor[sample_i]))
    print('small_vocab_kr {}번째 줄:  {}'.format(sample_i + 1, eng[sample_i]))

small_vocab_en 1번째 줄:  'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.
small_vocab_kr 1번째 줄:  Bible Coloring' is a coloring application that allows you to experience beautiful stories in the Bible.
small_vocab_en 2번째 줄:  씨티은행에서 일하세요?
small_vocab_kr 2번째 줄:  Do you work at a City bank?


In [11]:
kor_counter = collections.Counter([word for sentence in kor for word in sentence.split()])
eng_counter = collections.Counter([word for sentence in eng for word in sentence.split()])

print('{}개의 한국어 단어'.format(len([word for sentence in eng for word in sentence.split()])))
print('{}개의 유일한 한국어 단어'.format(len(kor_counter)))
print('가장 출현빈도가 높은 10개의 한국어 단어: ')
print('"' + '" "'.join(list(zip(*kor_counter.most_common(10)))[0]) + '"')
print()

print('{}개의 영어단어'.format(len([word for sentence in eng for word in sentence.split()])))
print('{}개의 유일한 영어단어'.format(len(eng_counter)))
print('가장 출현빈도가 높은 10개의 영어 단어: ')
print('"' + '" "'.join(list(zip(*eng_counter.most_common(10)))[0]) + '"')

4271602개의 한국어 단어
213041개의 유일한 한국어 단어
가장 출현빈도가 높은 10개의 한국어 단어: 
"나는" "수" "저는" "이" "있습니다." "내가" "우리는" "그" "있는" "당신이"

4271602개의 영어단어
79176개의 유일한 영어단어
가장 출현빈도가 높은 10개의 영어 단어: 
"the" "I" "to" "a" "you" "is" "and" "of" "in" "for"


#### Tokenizing

In [12]:
def tokenize(x):
  """
  Tokenize x
  :param x: List of sentences/strings to be tokenized
  :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
  """
  # TODO: Implement
  # tokenizer를 학습하고, 데이터를 bow로 만들어주세요

  tknzer = Tokenizer()
  tknzer.fit_on_texts(x)
  data = tknzer.texts_to_sequences(x) # data = result of tokenizer

  return data, tknzer

# Tokenize Example output
text_sentences = [
    '"Bible Coloring"은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.',
    '씨티은행에서 일하세요?',
    '푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'bible': 1, 'coloring': 2, '은': 3, '성경의': 4, '아름다운': 5, '이야기를': 6, '체험': 7, '할': 8, '수': 9, '있는': 10, '컬러링': 11, '앱입니다': 12, '씨티은행에서': 13, '일하세요': 14, '푸리토의': 15, '베스트셀러는': 16, '해외에서': 17, '입소문만으로': 18, '4차': 19, '완판을': 20, '기록하였다': 21}

Sequence 1 in x
  Input:  "Bible Coloring"은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.
  Output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Sequence 2 in x
  Input:  씨티은행에서 일하세요?
  Output: [13, 14]
Sequence 3 in x
  Input:  푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.
  Output: [15, 16, 17, 18, 19, 20, 21]


#### Padding

In [13]:
def pad(x, length=None):
    if length == None: 
        length = max(map(len, x))
        padded = pad_sequences(x, maxlen=length, padding='post')
    else:
        padded = pad_sequences(x, maxlen=length, padding='post')

    return np.array(padded) # sequence를 패딩하는 방법을 검색하여 사용하세요. 

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1)) # 내가 원하는 결과
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 1  2  3  4  5  6  7  8  9 10 11 12]
  Output: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Sequence 2 in x
  Input:  [13 14]
  Output: [13 14  0  0  0  0  0  0  0  0  0  0]
Sequence 3 in x
  Input:  [15 16 17 18 19 20 21]
  Output: [15 16 17 18 19 20 21  0  0  0  0  0]


In [14]:
def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_kor_sentences, preproc_eng_sentences, kor_tokenizer, eng_tokenizer =\
    preprocess(kor, eng)
    
max_kor_sequence_length = preproc_kor_sentences.shape[1]
max_eng_sequence_length = preproc_eng_sentences.shape[1]
kor_vocab_size = len(kor_tokenizer.word_index)
eng_vocab_size = len(eng_tokenizer.word_index)

print('Data Preprocessed')
print("최대 한국어 문장 길이:", max_kor_sequence_length)
print("최대 영어 문장 길이:", max_eng_sequence_length)
kor_vocab_size += 1
eng_vocab_size += 1
print("한국어 어휘 사전 크기:", kor_vocab_size)
print("영어 어휘 사전 크기:", eng_vocab_size)

Data Preprocessed
최대 한국어 문장 길이: 30
최대 영어 문장 길이: 53
한국어 어휘 사전 크기: 202031
영어 어휘 사전 크기: 40002
